In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras. preprocessing.sequence import pad_sequences
print(tf.__version__)

from sklearn.model_selection import train_test_split

2.4.1


In [2]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import collections

In [3]:
from google.colab import drive
drive.mount('/content/drive')

try:
  test = pd.read_csv('/content/drive/MyDrive/Data Science/test.csv')
  train = pd.read_csv('/content/drive/MyDrive/Data Science/train.csv')
except:
  test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/scl-2021-ds/test.csv')
  train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/scl-2021-ds/train.csv')
print(train.head())
print(test.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   id  ...                                 POI/street
0   0  ...  /jl kapuk timur delta sili iii lippo cika
1   1  ...                                          /
2   2  ...                                     /siung
3   3  ...                                 toko dita/
4   4  ...                             /jl. orde baru

[5 rows x 3 columns]
   id                                    raw_address
0   0          s. par 53 sidanegara 4 cilacap tengah
1   1          angg per, baloi indah kel. lubuk baja
2   2                          asma laun, mand imog,
3   3  ud agung rej, raya nga sri wedari karanganyar
4   4                     cut mutia, 35 baiturrahman


In [4]:
print(train['raw_address'].sample(20))

296515    tegal gundil pami 5, 15 rt 7 6 bogor utara - kota
67561             kali baru timur #, kali baru medan satria
81292                       sun angit 156 30752 babat toman
82194     perumahan wahana cikarang, blok b 2 no 25 suka...
225176      rio motor rs fatma, cipete utara kebayoran baru
77752                       pem poris plawad utara cipondoh
1868        raya cibu bog 392 gunungbatu bogor barat - kota
31007                    klinik al-i, raya bayongbong ciela
19669                                         rth bajulmati
240438                             sumur gayam, kapas kapas
49689                      k 33 ulujami rt 9 4 pesanggrahan
55227     globe surya udara pt, benda barat 2, kaliabang...
38219                    tpa an nur, kemuningsari lor panti
288403        pengadegan pan sela vii 28-30 rt 7 5 pancoran
216335    tamalanrea jaya peri kemerd viii bg.91 rt 5 rw...
111617    tanjung duren selatan gg. iii 8 rt 3 2 grogol ...
151639           mushallina tai, raya ja

In [5]:
# creating regex to split raw_address into words
my_regex = '\s|,\s|,|\.|:'

# POI

## Splitting Data - POI

In [6]:
raw_df_withPOI = train[train['POI/street'].str.contains('.+(?=/)')==True]
raw_df_withPOI['POI/street'] = raw_df_withPOI['POI/street'].str.extract(r'(.+)(?=/)')
raw_df_withPOI.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,raw_address,POI/street
3,3,"toko dita, kertosono",toko dita
5,5,"raya samb gede, 299 toko bb kids",toko bb kids
10,10,"cikahuripan sd neg boj 02 klap boj, no 5 16877",sd negeri bojong 02
11,11,"yaya atohar,",yayasan atohariyah
15,15,"kampung.gudang areng,desa:anyer, kecamatan:any...",gudang areng


In [7]:
# turning raw_address and POI/street columns into lists of strings; discarding id and index column
list_of_addresses_withPOI = [*raw_df_withPOI['raw_address']]
print(list_of_addresses_withPOI[:5])

list_of_POI = [*raw_df_withPOI['POI/street']]

['toko dita, kertosono', 'raya samb gede, 299 toko bb kids', 'cikahuripan sd neg boj 02 klap boj, no 5 16877', 'yaya atohar,', 'kampung.gudang areng,desa:anyer, kecamatan:anyar, kabupaten: serang, belakang bca anyar']


## Train Test Split - POI

In [8]:
# train_test_split to get training and testing datasets
train_addresses_withPOI, test_addresses_withPOI, train_labels_POI, test_labels_POI = train_test_split(
list_of_addresses_withPOI, list_of_POI, test_size=0.20, random_state=42)

print(len(train_addresses_withPOI))
print(len(train_labels_POI))

97192
97192


In [9]:
print(test_labels_POI[:5])

['toko kelontong siantar 21', 'perumahan graha indah', 'ruko pgm', 'cluster acacia', 'perumnas benteng']


## Creating labels_binary_POI

In [10]:
max_length_address = 25   #len(max(raw_df_withPOI['raw_address']))
max_length_POI = 10       #len(max(raw_df_withPOI['POI/street']))

In [11]:
# # testing labels_binary_POI with 5 rows of raw_df_withPOI.shape
# labels_binary_POI = [[]]*5 # *raw_df_withPOI.shape[0]
# print(len(raw_df_withPOI))
# print(raw_df_withPOI.shape[0])

# # for each word in 'raw_address', match it to words in 'POI/street' column.
# # if there's a match, value 1; if no match, value 0
# for i in range(len(raw_df_withPOI.iloc[:5])):
#   label_binary = [0 for _ in range(23)]
#   words = re.split(my_regex, raw_df_withPOI['POI/street'].iloc[i])
#   address = re.split(my_regex, raw_df_withPOI['raw_address'].iloc[i])

#   for word in words:
#     for j, ad in enumerate(address):
#       if ad in word:
#         label_binary[j] = 1
#         break

#   labels_binary_POI[i] = label_binary
 
# labels_binary_POI = np.array(labels_binary_POI)
# print(labels_binary_POI[:5])

In [12]:
# # finding out why 5th row has anomaly / wrong answer
# words = re.split(my_regex, raw_df_withPOI['raw_address'].iloc[4])
# print(words[8]) # check if the 9th position is whitespace (which is indicated as '1' in binary label)
# for word in words:
#   print(word)


In [13]:
dictionary_POI = {}

In [14]:
# binary labels for training set
number_of_binary_labels = len(train_addresses_withPOI)
print(len(train_addresses_withPOI))
labels_binary_POI = [[]]*number_of_binary_labels


# for each word in 'raw_address', match it to words in 'POI/street' column.
# if there's a match, value 1; if no match, value 0
for i in range(number_of_binary_labels):
  label_binary = [0 for _ in range(max_length_address)]
  words = re.split(my_regex, train_labels_POI[i])
  address = re.split(my_regex, train_addresses_withPOI[i])

  for word in words:
    for j, ad in enumerate(address):
      if word.startswith(ad):
        if (ad != word) & ((word in dictionary_POI.values())==False):
          dictionary_POI[ad] = word
        if j<max_length_address:
          label_binary[j] = 1
          break

  labels_binary_POI[i] = label_binary
 
labels_binary_POI = np.array(labels_binary_POI)
print(labels_binary_POI[:5])
print(dict(list(dictionary_POI.items())[0:5]))
print(len(dictionary_POI))

97192
[[1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
{'swa': 'swasta', 'bub': 'bubble', 'apo': 'apotek', 'lest': 'lestari', 'sen': 'sendal'}
8945


In [15]:
#binary labels for validation set
number_of_binary_labels = len(test_addresses_withPOI)
test_labels_binary_POI = [[]]*number_of_binary_labels


# for each word in 'raw_address', match it to words in 'POI/street' column.
# if there's a match, value 1; if no match, value 0
for i in range(len(test_addresses_withPOI)):
  label_binary = [0 for _ in range(max_length_address)]
  words = re.split(my_regex, test_labels_POI[i])
  address = re.split(my_regex, test_addresses_withPOI[i])

  for word in words:
    for j, ad in enumerate(address):
      if word.startswith(ad):
        if (ad != word) & ((word in dictionary_POI.values())==False):
          dictionary_POI[ad] = word
        if j<max_length_address:
          label_binary[j] = 1
          break

  test_labels_binary_POI[i] = label_binary
 
test_labels_binary_POI = np.array(test_labels_binary_POI)
print(test_labels_binary_POI[:5])
print(len(dictionary_POI))

[[0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
10200


In [16]:
print(len(dictionary_POI))

def filter_dict(mydict):
    filter_dict_num = {k: mydict[k] for k in sorted(mydict.keys()) if not any(map(str.isdigit, k))}
    filter_dict_len = {k: filter_dict_num[k] for k in sorted(filter_dict_num.keys()) if len(k)>0}
    return filter_dict_len

filtered_dict_POI = filter_dict(dictionary_POI)
print(len(filtered_dict_POI))

10200
10087


## Tokenizer and Padding - POI

In [17]:
# plotting distribution of length of raw_addresses and of POI
# length_of_addresses = [len(re.split('\s|, |\.|:', address)) for address in train_addresses_withPOI]
# print(length_of_addresses[:5])
# print(max(length_of_addresses))
# print(len(train_addresses_withPOI))

# A = collections.Counter(length_of_addresses)
# plt.bar(A.keys(), A.values())
# plt.show()

# length_of_POI = [len(re.split('\s|, |\.|:', POI)) for POI in train_labels_POI]
# print(length_of_POI[:5])
# print(max(length_of_POI))

# P = collections.Counter(length_of_POI)
# plt.bar(P.keys(), P.values())
# plt.show()

In [18]:
# vocab_size = 200000
embedding_dim = 64
trunc_type = 'post'
padding = 'post'
oov_tok="<OOV>"

print(type(train_addresses_withPOI))
sentences_to_tokenize = train_addresses_withPOI.copy()

print(type(sentences_to_tokenize))
# train_POI_sentences = []

print(len(train_addresses_withPOI))

sentences_to_tokenize.append(train_labels_POI)
print(len(train_addresses_withPOI))
print(sentences_to_tokenize[:5])


# sentences_to_tokenize = train_addresses.append(train_POI_sentences)
# print(sentences_to_tokenize[:5])
# for row in train_labels_POI:
#   train_addresses.append(row)
# print(sentences_to_tokenize[-5:])


# for row in test_labels_POI:
#   test_sentences.append(row)
# print(test_sentences[-5:])

<class 'list'>
<class 'list'>
97192
97192
['mi swa islamiyah syafiiyah,', 'es bub blend maak nyoes, salak,', 'apo pasar pagi, pami, 20 kejaksan', 'raya cibatu-li, tk islam al hikmah, cibatu', 'isur iv, raya ciwi,']


In [19]:
if len(train_addresses_withPOI) != len(train_labels_POI):
  print(len(train_addresses_withPOI))
  print(len(train_labels_POI))

In [20]:
# Tokenizing train_addresses and train_POI

tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences_to_tokenize)

word_index = tokenizer.word_index
print(len(word_index))
total_words = len(word_index)+1

def tokenize_and_pad(data):
  data_sequences = tokenizer.texts_to_sequences(data)
  data_padded = pad_sequences(data_sequences, maxlen=max_length_address, truncating=trunc_type, padding=padding)
  return data_padded

#Padding train_addresses and test_addresses
train_padded = tokenize_and_pad(train_addresses_withPOI)
print(train_padded[0])
print(train_padded.shape)

test_padded = tokenize_and_pad(test_addresses_withPOI)
print(test_padded[0])
print(test_padded.shape)

131657
[  481   558  4941 33211     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0]
(97192, 25)
[ 915    6 1244 3111  194  234   11    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]
(24299, 25)


In [21]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(train_padded[1]))
print(train_addresses_withPOI[1])

es bub blend maak nyoes salak ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
es bub blend maak nyoes, salak,


# Subword Tokenization
too advanced, skipping

In [22]:
# #!pip install -q tensorflow_text_nightly
# !pip install -q tf-nightly
# !pip install tokenizers -q

# from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
# from tokenizers import BertWordPieceTokenizer

In [23]:
# # Initialize an empty BERT tokenizer
# tokenizer = BertWordPieceTokenizer(
#   clean_text=False,
#   handle_chinese_chars=False,
#   strip_accents=False,
#   lowercase=True,
# )

# # prepare text files to train vocab on them
# test_string = ['/content/test.txt']
# #[sentences_to_tokenize]
# #

# # train BERT tokenizer
# tokenizer.train(
#   test_string,
#   vocab_size=20,
#   min_frequency=2,
#   show_progress=True,
#   special_tokens=['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'],
#   limit_alphabet=1000,
#   wordpieces_prefix="##"
# )

# # save the vocab
# tokenizer.save('/content/bert-vocab.txt')

# # create a BERT tokenizer with trained vocab
# vocab = '/content/bert-vocab.txt'
# tokenizer = BertWordPieceTokenizer(vocab)

# # test the tokenizer with some text
# encoded = tokenizer.encode('...')
# print(encoded.tokens)

In [24]:
# #bert_tokenizer_params=dict(lower_case=True)
# reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

# bert_vocab_args = dict(
#     # The target vocabulary size
#     vocab_size = 97192,
#     # Reserved tokens that must be included in the vocabulary
#     reserved_tokens=reserved_tokens,
#     # Arguments for `text.BertTokenizer`
#     bert_tokenizer_params=dict(lower_case=True),
#     # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
#     learn_params={},
# )

# vocab = bert_vocab.bert_vocab_from_dataset(
#     sentences_to_tokenize, 
#     **bert_vocab_args
# )


In [25]:
# Tokenize corpus (all words AI should learn) without vocab_size limit 

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(sentences_to_tokenize)

# word_index = tokenizer.word_index
# print(len(word_index))


## Model building - POI

In [26]:
# basic model - no Bidirectional, LSTM, etc.
model_POI = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, embedding_dim, input_length=max_length_address),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(max_length_address, activation='sigmoid')
])
model_POI.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_POI.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 64)            8426112   
_________________________________________________________________
bidirectional (Bidirectional (None, 25, 32)            10368     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                16640     
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 25)                1625      
Total params: 8,458,905
Trainable params: 8,458,905
Non-trainable params: 0
______________________________________________

In [27]:
if train_padded.shape[0] != labels_binary_POI.shape[0]:
  print('Training addresses and labels have different number of rows')

In [ ]:
num_epochs = 3

history = model_POI.fit(train_padded, labels_binary_POI, epochs=num_epochs, validation_data=(test_padded, test_labels_binary_POI))

Epoch 1/10
3038/3038 [==============================] - 266s 85ms/step - loss: 0.1523 - accuracy: 0.3937 - val_loss: 0.0745 - val_accuracy: 0.4404
Epoch 2/10
3038/3038 [==============================] - 259s 85ms/step - loss: 0.0596 - accuracy: 0.4346 - val_loss: 0.0629 - val_accuracy: 0.4979
Epoch 3/10
3038/3038 [==============================] - 260s 86ms/step - loss: 0.0409 - accuracy: 0.4686 - val_loss: 0.0639 - val_accuracy: 0.4639
Epoch 4/10
3038/3038 [==============================] - 261s 86ms/step - loss: 0.0294 - accuracy: 0.5084 - val_loss: 0.0679 - val_accuracy: 0.4717
Epoch 5/10
3038/3038 [==============================] - 259s 85ms/step - loss: 0.0231 - accuracy: 0.5242 - val_loss: 0.0744 - val_accuracy: 0.5177
Epoch 6/10
3038/3038 [==============================] - 255s 84ms/step - loss: 0.0184 - accuracy: 0.5321 - val_loss: 0.0813 - val_accuracy: 0.6123
Epoch 7/10
3038/3038 [==============================] - 254s 84ms/step - loss: 0.0146 - accuracy: 0.5722 - val_loss: 0

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

## model_POI.predict

In [ ]:
print(type(test))
live_padded = tokenize_and_pad(test['raw_address'])
print(live_padded.shape)

# live_to_predict = test_padded[:5]
# live_prediction_POI = model_POI.predict(live_to_predict)

def predict_and_convert_to_words(live_to_predict, raw_address, threshold, dictionary, model):
  live_prediction = model.predict(live_to_predict)
  number_of_live_labels = len(live_to_predict)
  # print(number_of_live_labels)

  live_labels = [[]]*number_of_live_labels
  
  # decode live_labels to regular words
  for i in range(number_of_live_labels):
    live_label = ''
    address = re.split(my_regex, raw_address.iloc[i])
  
    for id, j in enumerate(live_prediction[i]):
      if (j > threshold) & (id < len(address)):
        if address[id] in dictionary:
          # print(id, j, address[id])
          new_string = dictionary.get(address[id])
          live_label += new_string
          live_label += ' '
        else:
          # print(id, j, address[id])
          live_label += address[id]
          live_label += ' '
    # print(live_label_POI)
    live_label = live_label.rstrip()
    live_labels[i] = live_label
  return live_labels

live_labels_POI = predict_and_convert_to_words(live_padded, test['raw_address'], threshold=0.5, filtered_dict_POI, model_POI)
print(live_labels_POI[:5])
print(test['raw_address'][:5])

# if want to run test, will get error as train_addresses_withPOI is not a DataFrame
# test = predict_and_convert_to_words(test_padded[:5], train_addresses_withPOI, filtered_dict_POI, model_POI)
# print(test)

In [ ]:
# print(type(test))
# live_padded = tokenize_and_pad(test['raw_address'])
# print(live_padded.shape)

# # live_to_predict = test_padded[:5]
# # live_prediction_POI = model_POI.predict(live_to_predict)


# live_prediction = model_POI.predict(live_padded)
# number_of_live_labels = len(live_padded)
# print(number_of_live_labels)

# live_labels = [[]]*number_of_live_labels
  
# # decode live_labels to regular words
# for i in range(number_of_live_labels):
#   live_label = ''
#   address = re.split(my_regex, test['raw_address'].iloc[i])
  
#   for id, j in enumerate(live_prediction[i]):
#     if (j > 0.5) & (id < len(address)):
#       if address[id] in filtered_dict_POI:
#         # print(id, j, address[id])
#         new_string = filtered_dict_POI.get(address[id])
#         live_label += new_string
#         live_label += ' '
#       else:
#         # print(id, j, address[id])
#         live_label += address[id]
#         live_label += ' '
#   live_label = live_label.rstrip()

#   live_labels[i] = live_label


# print(live_labels[:5])
# print(test['raw_address'][:5])
# print(live_prediction[4])

# Street (ST)

## Splitting Data - ST

In [ ]:
raw_df_withST = train[train['POI/street'].str.contains('(?<=/).+')==True]
raw_df_withST['POI/street'] = raw_df_withST['POI/street'].str.extract(r'(?<=/)(.+)')
raw_df_withST.head()

In [ ]:
# turning raw_address and POI/street columns into lists of strings; discarding id and index column
list_of_addresses_withST = [*raw_df_withST['raw_address']]
print(list_of_addresses_withST[:5])

list_of_ST = [*raw_df_withST['POI/street']]

## Train Test Split - ST

In [ ]:
# train_test_split to get training and testing datasets
train_addresses_withST, test_addresses_withST, train_labels_ST, test_labels_ST = train_test_split(
list_of_addresses_withST, list_of_ST, test_size=0.20, random_state=42)

print(len(train_addresses_withST))
print(len(train_labels_ST))

## Creating labels_binary_ST

In [ ]:
# plotting distribution of length of raw_addresses and of POI
length_of_addresses_ST = [len(re.split('\s|, |\.|:', address)) for address in train_addresses_withST]
print(length_of_addresses_ST[:5])
print(max(length_of_addresses_ST))

A = collections.Counter(length_of_addresses_ST)
plt.bar(A.keys(), A.values())
plt.show()

length_of_ST = [len(re.split('\s|, |\.|:', ST)) for ST in train_labels_ST]
print(length_of_ST[:5])
print(max(length_of_ST))

P = collections.Counter(length_of_ST)
plt.bar(P.keys(), P.values())
plt.show()

In [ ]:
max_length_address = 25   #len(max(raw_df_withST['raw_address']))
max_length_ST = 10       #len(max(raw_df_withST['POI/street']))

In [ ]:
dictionary_ST = {}

In [ ]:
# binary labels for training set
number_of_binary_labels = len(train_addresses_withST)
print(len(train_addresses_withST))
labels_binary_ST = [[]]*number_of_binary_labels


# for each word in 'raw_address', match it to words in 'POI/street' column.
# if there's a match, value 1; if no match, value 0
for i in range(number_of_binary_labels):
  label_binary = [0 for _ in range(max_length_address)]
  words = re.split(my_regex, train_labels_ST[i])
  address = re.split(my_regex, train_addresses_withST[i])

  for word in words:
    for j, ad in enumerate(address):
      if word.startswith(ad):
        if (ad != word) & ((word in dictionary_ST.values())==False):
          dictionary_ST[ad] = word
        if j<max_length_address:
          label_binary[j] = 1
          break

  labels_binary_ST[i] = label_binary
 
labels_binary_ST = np.array(labels_binary_ST)
print(labels_binary_ST[:5])
print(train_addresses_withST[:5])
print(train_labels_ST[:5])

In [ ]:
#binary labels for validation set
number_of_binary_labels = len(test_addresses_withST)
test_labels_binary_ST = [[]]*number_of_binary_labels


# for each word in 'raw_address', match it to words in 'POI/street' column.
# if there's a match, value 1; if no match, value 0
for i in range(len(test_addresses_withST)):
  label_binary = [0 for _ in range(max_length_address)]
  words = re.split(my_regex, test_labels_ST[i])
  address = re.split(my_regex, test_addresses_withST[i])

  for word in words:
    for j, ad in enumerate(address):
      if word.startswith(ad):
        if (ad != word) & ((word in dictionary_ST.values())==False):
          dictionary_ST[ad] = word
        if j<max_length_address:
          label_binary[j] = 1
          break

  test_labels_binary_ST[i] = label_binary
 
test_labels_binary_ST = np.array(test_labels_binary_ST)
print(test_labels_binary_ST[:5])
print(len(dictionary_ST))

In [ ]:
print(len(dictionary_ST))
filtered_dict_ST = filter_dict(dictionary_ST)
print(len(filtered_dict_ST))

## Tokenizer and Padding - ST

In [ ]:
embedding_dim = 64
trunc_type = 'post'
padding = 'post'
oov_tok="<OOV>"

print(type(train_addresses_withST))
sentences_to_tokenize = train_addresses_withST.copy()

print(type(sentences_to_tokenize))
# train_ST_sentences = []

print(len(train_addresses_withST))

sentences_to_tokenize.append(train_labels_ST)
print(len(train_addresses_withST))
print(sentences_to_tokenize[:5])


# sentences_to_tokenize = train_addresses.append(train_ST_sentences)
# print(sentences_to_tokenize[:5])
# for row in train_labels_ST:
#   train_addresses.append(row)
# print(sentences_to_tokenize[-5:])


# for row in test_labels_ST:
#   test_sentences.append(row)
# print(test_sentences[-5:])

In [ ]:
if len(train_addresses_withPOI) != len(train_labels_POI):
  print(len(train_addresses_withPOI))
  print(len(train_labels_POI))

In [ ]:
# Tokenizing train_addresses and train_ST

tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences_to_tokenize)

word_index = tokenizer.word_index
print(len(word_index))
total_words = len(word_index)+1

#Padding train_addresses, train_ST, test_addresses
train_padded = tokenize_and_pad(train_addresses_withST)
print(train_padded[0])
print(train_padded.shape)

test_padded = tokenize_and_pad(test_addresses_withST)

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(train_padded[1]))
print(train_addresses_withST[1])

## Model building - ST

In [ ]:
# basic model - no Bidirectional, LSTM, etc.
model_ST = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, embedding_dim, input_length=max_length_address),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(max_length_address, activation='sigmoid')
])
model_ST.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_ST.summary()

In [ ]:
if train_padded.shape[0] != labels_binary_ST.shape[0]:
  print('Training addresses and labels have different number of rows')

In [ ]:
num_epochs = 10

history = model_ST.fit(train_padded, labels_binary_ST, epochs=num_epochs, validation_data=(test_padded, test_labels_binary_ST))

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

## model_ST.predict

In [ ]:
live_padded_ST = tokenize_and_pad(test['raw_address'])
print(live_padded_ST.shape)

# live_to_predict = test_padded[:5]
# live_prediction_ST = model_ST.predict(live_to_predict)


# number_of_live_labels = len(live_to_predict)
# print(number_of_live_labels)

# live_labels_ST = [[]]*number_of_live_labels

# # decode live_labels_ST to regular words
# for i in range(number_of_live_labels):
#   live_label_ST = ''
#   address = re.split(my_regex, test_addresses_withST[i])
#   # word = re.split(my_regex, address)
  
#   for id, j in enumerate(live_prediction_ST[i]):
#     if (j > 0.5) & (id <= len(address)):
#       if address[id] in dictionary_ST:
#         # print(id, j, address[id])
#         new_string = dictionary_ST.get(address[id])
#         live_label_ST += new_string
#         live_label_ST += ' '
#       else:
#         # print(id, j, address[id])
#         live_label_ST += address[id]
#         live_label_ST += ' '
#   # print(live_label_ST)

#   live_labels_ST[i] = live_label_ST

# print(live_labels_ST[:5])
# print(test_labels_ST[:5])
# print(test_addresses_withST[:5])

In [ ]:
live_labels_ST = predict_and_convert_to_words(live_padded_ST, test['raw_address'], threshold=0.5, filtered_dict_ST, model_ST)
print(live_labels_ST[:5])
print(test['raw_address'][:5])

# Combine results of POI and ST - POI/Street column

In [ ]:
print('live_labels_ST is type: ' + str(type(live_labels_ST)))
print('live_labels_POI is type: ' + str(type(live_labels_POI)))

# check if live_labels_ST and _POI have same number of elements
if len(live_labels_ST) != len(live_labels_POI):
  print('live_labels_ST and _POI have different number of elements')

In [ ]:
test['POI'] = live_labels_POI
test['Street'] = live_labels_ST

print(test.head())

test['POI/Street'] = test.POI.cat(test.Street, sep='/')
print(test.head())

submission = test[['id', 'POI/Street']]
print(submission)

submission.to_csv('submission.csv', index=False)